In [ ]:
from IPython.display import clear_output
# !pip install wandb
clear_output()


In [ ]:
!pip install datasets
clear_output()

In [ ]:
!pip install peft
clear_output()

In [ ]:
!pip install bs4
clear_output()

In [ ]:
!wget https://raw.githubusercontent.com/Puspita-111/Suven-consultants-and-technology/main/contractions.py
!wget https://raw.githubusercontent.com/Puspita-111/Suven-consultants-and-technology/main/text_normalizer.py

--2024-02-24 16:43:04--  https://raw.githubusercontent.com/Puspita-111/Suven-consultants-and-technology/main/contractions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3110 (3.0K) [text/plain]
Saving to: ‘contractions.py’

contractions.py     100%[===================>]   3.04K  --.-KB/s    in 0s      

2024-02-24 16:43:04 (45.6 MB/s) - ‘contractions.py’ saved [3110/3110]

--2024-02-24 16:43:04--  https://raw.githubusercontent.com/Puspita-111/Suven-consultants-and-technology/main/text_normalizer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respons

In [ ]:

import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
# import wandb
import os
import re

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
# from trl import SFTTrainer

from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata


import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import string

tokenizer = ToktokTokenizer()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
!pip install nltk
clear_output()

In [ ]:
!pip install punkt
clear_output()

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from bs4 import BeautifulSoup

train_dataset = pd.read_csv('/content/train.csv')
test_dataset = pd.read_csv('/content/test.csv')

train_dataset['label'] = [1 if x=="positive" else 0 for x in train_dataset['sentiment'] ]
test_dataset['label'] = [1 if x=="positive" else 0 for x in test_dataset['sentiment'] ]

train_dataset = train_dataset.drop('sentiment', axis=1)
test_dataset = test_dataset.drop('sentiment', axis=1)

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):

    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    return text

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words and token not in string.punctuation]
    return ' '.join(tokens)


test_dataset['review'] = test_dataset['review'].apply(strip_html_tags)
train_dataset['review'] = train_dataset['review'].apply(strip_html_tags)

test_dataset['review'] = test_dataset['review'].apply(remove_accented_chars)
train_dataset['review'] = train_dataset['review'].apply(remove_accented_chars)

test_dataset['review'] = test_dataset['review'].apply(expand_contractions)
train_dataset['review'] = train_dataset['review'].apply(expand_contractions)

test_dataset['review'] = test_dataset['review'].apply(remove_special_characters)
train_dataset['review'] = train_dataset['review'].apply(remove_special_characters)

test_dataset['review'] = test_dataset['review'].apply(preprocess_text)
train_dataset['review'] = train_dataset['review'].apply(preprocess_text)



train_texts, train_labels = train_dataset['review'], train_dataset['label']
test_texts, test_labels = test_dataset['review'], test_dataset['label']

train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)

from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

train_texts.reset_index(drop=True, inplace=True)
val_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

import torch

class IMDBdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure all encodings and labels have consistent lengths
        assert all(len(val) == len(self.labels) for val in self.encodings.values()), "Encodings and labels lengths do not match."

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = IMDBdataset(train_encodings, train_labels)
test_dataset = IMDBdataset(test_encodings,test_labels)
val_dataset = IMDBdataset(val_encodings, val_labels)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to(device=device)

<ipython-input-20-22a1958b41f1>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
raw_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
raw_model = raw_model.to(device=device)
raw_model.eval()


# Store predictions and actual labels
raw_predictions = []
raw_actuals = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move tensors to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Only needed if you're also evaluating performance

        outputs = raw_model(input_ids, attention_mask=attention_mask)

        # Assuming you're doing classification and want the highest probability class
        logits = outputs[0]
        raw_predicted_labels = torch.argmax(logits, dim=1)
        raw_predictions.extend(raw_predicted_labels.cpu().numpy())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 20000/20000 [06:34<00:00, 50.76it/s]


In [ ]:
raw_actual_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        raw_labels = batch['labels'].to(device)  # Assuming labels are on the same device
        raw_actual_labels.extend(raw_labels.cpu().numpy())

100%|██████████| 20000/20000 [00:07<00:00, 2608.27it/s]


In [ ]:
raw_correct_predictions = sum(p == a for p, a in zip(raw_predictions, raw_actual_labels))
raw_accuracy = raw_correct_predictions / len(raw_predictions)
print(f"Accuracy: {raw_accuracy * 100:.2f}%")

Accuracy: 52.12%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
# Ensure predictions and actual_labels are numpy arrays or compatible formats
precision = precision_score(raw_actual_labels, raw_predictions)
recall = recall_score(raw_actual_labels, raw_predictions)
f1 = f1_score(raw_actual_labels, raw_predictions)
conf_matrix = confusion_matrix(raw_actual_labels, raw_predictions)
mcc = matthews_corrcoef(raw_actual_labels, raw_predictions)

# ROC-AUC score requires probability scores of the positive class, which might need model.predict_proba() or equivalent
# If your model outputs probabilities, you can use:
# roc_auc = roc_auc_score(actual_labels, prediction_probabilities)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Matthews Correlation Coefficient: {mcc:.2f}")
# print(f"ROC-AUC Score: {roc_auc:.2f}")  # Uncomment if you have probability predictions


Precision: 0.55
Recall: 0.28
F1 Score: 0.37
Confusion Matrix:
[[7566 2369]
 [7206 2859]]
Matthews Correlation Coefficient: 0.05


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)

# Fit and transform the training data to create TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)

# Transform the test data to TF-IDF vectors
X_test_tfidf = tfidf_vectorizer.transform(test_texts)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef

# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, train_labels)

# Predict on the test set
nb_predictions = nb_classifier.predict(X_test_tfidf)

# Compute metrics
nb_accuracy = accuracy_score(test_labels, nb_predictions)
nb_precision = precision_score(test_labels, nb_predictions, average='binary')
nb_recall = recall_score(test_labels, nb_predictions, average='binary')
nb_f1 = f1_score(test_labels, nb_predictions, average='binary')
nb_conf_matrix = confusion_matrix(test_labels, nb_predictions)
nb_mcc = matthews_corrcoef(test_labels, nb_predictions)

print("Naive Bayes Metrics:")
print(f"Accuracy: {nb_accuracy:.4f}")
print(f"Precision: {nb_precision:.4f}")
print(f"Recall: {nb_recall:.4f}")
print(f"F1 Score: {nb_f1:.4f}")
print(f"Confusion Matrix: \n{nb_conf_matrix}")
print(f"Matthews Correlation Coefficient: {nb_mcc:.4f}")


Naive Bayes Metrics:
Accuracy: 0.8574
Precision: 0.8633
Recall: 0.8515
F1 Score: 0.8573
Confusion Matrix: 
[[8578 1357]
 [1495 8570]]
Matthews Correlation Coefficient: 0.7149


In [ ]:
test_dataset.head()

AttributeError: 'IMDBdataset' object has no attribute 'head'